# Stepwise Regression Forward 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# yahoo finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2017-01-01'
end = '2017-12-31'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,11.42,11.65,11.02,11.43,11.43,55182000
2017-01-04,11.45,11.52,11.24,11.43,11.43,40781200
2017-01-05,11.43,11.69,11.23,11.24,11.24,38855200
2017-01-06,11.29,11.49,11.11,11.32,11.32,34453500
2017-01-09,11.37,11.64,11.31,11.49,11.49,37304800


In [3]:
# Create more data
# dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
# dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,-1)
# dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,-1)
# dataset['Return'] = dataset['Adj Close'].pct_change()
# dataset = dataset.dropna()
# dataset.head()

In [4]:
dataset = dataset.rename(index=str, columns={"Adj Close":'Adj_Close'})
dataset.head()

,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2017-01-03 00:00:00,11.42,11.65,11.02,11.43,11.43,55182000
2017-01-04 00:00:00,11.45,11.52,11.24,11.43,11.43,40781200
2017-01-05 00:00:00,11.43,11.69,11.23,11.24,11.24,38855200
2017-01-06 00:00:00,11.29,11.49,11.11,11.32,11.32,34453500
2017-01-09 00:00:00,11.37,11.64,11.31,11.49,11.49,37304800


In [5]:
dataset.shape

(251, 6)

In [6]:
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [7]:
model = forward_selected(dataset, 'Volume')

In [8]:
print(model.model.formula)

print(model.rsquared_adj)

Volume ~ High + Low + Close + Adj_Close + 1
0.6403456692998204


Another method of Stepwise Regression Forward

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [10]:
dataset = yf.download(symbol,start,end)

[*********************100%***********************]  1 of 1 completed


In [11]:
# Create more data
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,-1)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,-1)
dataset['Return'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Return
Date,,,,,,,,,,
2017-01-04,11.45,11.52,11.24,11.43,11.43,40781200,0,-1,-1,0.000000
2017-01-05,11.43,11.69,11.23,11.24,11.24,38855200,0,-1,1,-0.016623
2017-01-06,11.29,11.49,11.11,11.32,11.32,34453500,1,1,1,0.007117
2017-01-09,11.37,11.64,11.31,11.49,11.49,37304800,0,1,-1,0.015018
2017-01-10,11.55,11.63,11.33,11.44,11.44,29201600,1,-1,-1,-0.004352


In [12]:
X = dataset[['Open', 'High', 'Low', 'Close', 'Volume','Return']].values
# X = dataset.iloc[:, dataset.columns != 'Buy_Sell', 'Close']
Y = dataset['Buy_Sell'] # Cannot be Continuous

In [13]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

print('Training dataset shape:', X_train.shape, y_train.shape)
print('Testing dataset shape:', X_test.shape, y_test.shape)

Training dataset shape: (187, 6) (187,)
Testing dataset shape: (63, 6) (63,)


In [14]:
# Build RF classifier to use in feature selection
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

# Build step forward feature selection
sfs1 = sfs(clf,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    9.0s finished

[2022-06-15 18:52:31] Features: 1/5 -- score: 0.556330014224751[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.7s finished

[2022-06-15 18:52:37] Features: 2/5 -- score: 0.55049786628734[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.5s finished

[2022-06-15 18:52:42] Features: 3/5 -- score: 0.5125177809388336[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out

In [15]:
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[0, 1, 2, 3, 5]


In [16]:
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
# Build full model with selected features
clf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train[:, feat_cols], y_train)

y_train_pred = clf.predict(X_train[:, feat_cols])
print('Training accuracy on selected features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test[:, feat_cols])
print('Testing accuracy on selected features: %.3f' % acc(y_test, y_test_pred))

Training accuracy on selected features: 0.861
Testing accuracy on selected features: 0.460


In [19]:
# Build full model on ALL features, for comparison
clf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
print('Training accuracy on all features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test)
print('Testing accuracy on all features: %.3f' % acc(y_test, y_test_pred))

Training accuracy on all features: 0.872
Testing accuracy on all features: 0.460


In [20]:
y_pred = clf.predict(X_test)

In [21]:
print("Mean Absolute Error:", mean_squared_error(y_test, y_pred))
print("Coefficient of Determination:", r2_score(y_test, y_pred))


Mean Absolute Error: 2.1587301587301586
Coefficient of Determination: -1.1636363636363636
